In [ ]:
import sys
sys.path.append('/usr/local/lib/python2.7/dist-packages')
sys.path.append('/usr/local/lib/python3.5/dist-packages')
sys.path

In [ ]:
import create_dict
import torch
import os
import json
import utils
import numpy as np

# Clean Data

In [ ]:
#folder_name = '/data2'
#create_dict.clean_obj(folder_name)

# Generate Dataset

In [ ]:
#!pip3 freeze | grep PyWavefront

In [ ]:
#import pywavefront
#mesh2 = pywavefront.Wavefront('/home/.schramm/Documents/3d_retrieval/data/02747177/8bdea15ae5d929d0a2eb129d649f68cf/models/model_normalized.obj', collect_faces='True')


In [ ]:
input_folder = '/data'
suffix = '_test' #suffix will append data_trainxxxx.json
max_elements_per_class = 50
#create_dict.create_dictionary(input_folder, max_elements_per_class, suffix, points_per_object=2500)

# Initialize Network

In [ ]:
import SiameseNet

device = torch.device("cuda:0" if torch.cuda.torch.cuda.is_available() else "cpu")

batch_size = 64
net = SiameseNet.SiameseNet(batch_size, num_points=1500)
suffix = 'CT_2000p' # comment in if not coming from generating the dataset
#suffix = '_test'
#path_to_params = "models/autoencoder5_100epochs_BS16_CT_Gewichtung_text*1_shape*200_BS32.pt" # if file does not exist or is empty it starts from untrained and later saves to the file
path_to_params = "models/testtt.pt"

# shift to GPU if available

net.to(device)
working_dir = os.getcwd()
data_dir_train = os.path.join(working_dir, 'data_train'+suffix+'.json')
data_dir_val = os.path.join(working_dir, 'data_val'+suffix+'.json')
class_dir = os.path.join(working_dir, 'class_dict'+suffix+'.json')

path_to_hidden = str(path_to_params[:-3] + '_hidden.pt')
#if os.path.isfile(path_to_hidden):
#    net.hidden = torch.load(path_to_hidden)
if os.path.isfile(path_to_params):
    if os.stat(path_to_params).st_size != 0:
        net.load_state_dict(torch.load(path_to_params, map_location=device))  #Loads pretrained net if file exists and if not empty
else:
    open(path_to_params, "x") #Creates parameter file if it does not exist

In [ ]:
device

# Train Network

In [ ]:
#training parameters

#writer_suffix = 'autoencoder5_100epochs_BS16_CT_Gewichtung_text*1_shape*200_BS32'
writer_suffix = 'testrr'
margin = 0.5
num_epochs = 100
print_batch = 10
lr = 2e-3



In [ ]:
net = SiameseNet.train(net, num_epochs, margin, lr, print_batch, 
                       data_dir_train, data_dir_val, writer_suffix, path_to_params, working_dir, class_dir)

# Validation

In [ ]:
margin = 0.5
writer_suffix = 'testrr_Val_images'
metric=[]

# average metric of k runs
k=1
for i in range(k):
    metric.append(SiameseNet.val(net, margin, data_dir_val, writer_suffix, working_dir, class_dir, k=5,  images=True))

scores=np.asarray(metric)*100
scores2 = np.mean(scores, axis=0)
print('averaged scores are:')
#print(scores2)
#print('with standard deviation:')
var = np.std(scores, axis=0)
#print(var)
print()
print('rr@1:', np.round(scores2[0],4), '+-',np.round(var[0]*1.96, 4))
print('rr@5:', np.round(scores2[1],4), '+-',np.round(var[1]*1.96, 4))
print('rr@10:', np.round(scores2[2],4), '+-',np.round(var[2]*1.96, 4))
print('NDCG@5:', np.round(scores2[3],4), '+-',np.round(var[3]*1.96, 4))
print('NDCG@10:', np.round(scores2[4],4), '+-',np.round(var[4]*1.96, 4))

# Retrieval

In [ ]:
y_true, y_pred, ids, shape, description = SiameseNet.retrieval(net, data_dir_val, working_dir, print_nn=False)

In [ ]:
utils.retrieve_images(y_pred, ids, data_dir_val, class_dir, num_KNN=5, max_show=4, shuffle=False)

In [ ]:
sentence = 'a wooden craft chair'
#sentence = input("Enter sentence")
utils.retrieve_one_sentence(net, data_dir_val, working_dir, sentence, class_dir, y_pred , ids, shape, num_KNN=5,samples_pointcould=2500)

# Test

In [ ]:
rr1, rr5, rr10, ndcg5, ndcg10 = SiameseNet.test(net, margin, data_dir_val, working_dir)
print('retrieval performance measures:')
print('rr@1', np.round((rr1*100),4))
print('rr@5', np.round((rr5*100),4))
print('rr@10', np.round((rr10*100),4))
print('NDCG@5', np.round((ndcg5*100),4))
print('NDCG@10', np.round((ndcg10*100),4))
